In [ ]:
import re
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import os
import joblib
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score,roc_auc_score,accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, IsolationForest

from functools import reduce

from datetime import datetime
from datetime import datetime, timedelta
from time import time

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from datetime import datetime
from datetime import datetime, timedelta
from time import time
from timeit import default_timer as timer
#start = timer()
#end = timer()
#print(end - start) 

import random

In [ ]:
def under_sampling(original_features, original_labels):

    X = original_features
    y = original_labels
    undersample = RandomUnderSampler(sampling_strategy=0.8)
    X_under, y_under = undersample.fit_resample(X, y)
    malware_undersampled = X_under
    malware_undersampled['Label']=y_under
    print(f"Count after undersampling: \n{malware_undersampled['Label'].value_counts()}")
    print(f"Shape of dataset: \n{malware_undersampled.shape}")
    
    undersampled_features = malware_undersampled.drop(columns = ["Label"])
    undersampled_labels = malware_undersampled.Label

    return undersampled_features, undersampled_labels

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        title = 'Confusion matrix'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
  #  classes = unique_labels(y_true, y_pred)
    print('Confusion matrix')
    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
def poison_lf(labels,kpoison):    
    z_train=labels
    a=0;b=0; #just indicators 
    kpoison=0.5
    rate_poison=round(kpoison*y_train.shape[0]);
    for l in range(0,rate_poison):
        location=random.randint(0,31410)
        c=z_train[location];
        if (c==0):
            z_train[location]=1
            a=a+1 #label-flipped samples for label 0
        else:
            z_train[location]=0
            b=b+1 #label flipped samples for label 1    
    print(f"The number of data poisoned is the following: \n{a+b}\n Out of a total of: \n{z_train.shape[0]}")
    #a+b=how many samples have been label flipped
    return z_train

In [ ]:
train_set = pd.read_csv('train_set.csv')
test_set = pd.read_csv('test_set.csv')

In [ ]:
train_set.head()

In [ ]:
x_train = train_set.drop(columns = ['Label'])
y_train = train_set.Label
x_test = test_set.drop(columns = ['Label'])
y_test = test_set.Label

In [ ]:
print(f"The ORIGINAL label vector is the following: \n{y_train}")
#y_train 
print(f"Shape of LABEL VECTOR: \n{y_train.shape[0]}")

In [ ]:
poison=0.5 #proportion of samples to be poisoned
y_train=poison_lf(y_train,poison)
print(f"The POISONED label vector is the following: \n{y_train}")

In [ ]:
# Random Forest algorithm
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
y_predicted = rf.predict(x_test).astype(int)
plot_confusion_matrix(y_test, y_predicted, classes=unique_labels(y_test.astype(int), y_predicted), title='Confusion matrix')
plt.show()
print(f"The accuracy is: {accuracy_score(y_test, y_predicted)}")
print(f"The precision is: {precision_score(y_test, y_predicted)}")
print(f"The recall is: {recall_score(y_test, y_predicted)}")
print(f"The f1 score is: {f1_score(y_test, y_predicted)}")
print(f"The auc score is: {roc_auc_score(y_test, y_predicted)}")